In [1]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import asyncio
import nest_asyncio


2023-10-16 00:19:19.731964: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-16 00:19:19.732016: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-16 00:19:19.732036: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-16 00:19:19.737566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
#pip freeze

absl-py==2.0.0
annotated-types==0.6.0
anyio==3.7.1
asttokens==2.4.0
astunparse==1.6.3
backcall==0.2.0
cachetools==5.3.1
certifi==2023.7.22
charset-normalizer==3.3.0
click==8.1.7
comm==0.1.4
contourpy==1.1.1
cycler==0.12.1
debugpy==1.8.0
decorator==5.1.1
dm-tree==0.1.8
exceptiongroup==1.1.3
executing==2.0.0
fastapi==0.103.2
flatbuffers==23.5.26
fonttools==4.43.1
gast==0.5.4
google-auth==2.23.3
google-auth-oauthlib==1.0.0
google-pasta==0.2.0
grpcio==1.59.0
h11==0.14.0
h5py==3.10.0
httptools==0.6.0
idna==3.4
ipykernel==6.25.2
ipython==8.16.1
jedi==0.19.1
jupyter_client==8.3.1
jupyter_core==5.4.0
keras==2.14.0
keras-core==0.1.7
keras-tuner==1.4.4
kiwisolver==1.4.5
kt-legacy==1.0.5
libclang==16.0.6
Markdown==3.5
markdown-it-py==3.0.0
MarkupSafe==2.1.3
matplotlib==3.8.0
matplotlib-inline==0.1.6
mdurl==0.1.2
ml-dtypes==0.2.0
namex==0.0.7
nest-asyncio==1.5.8
numpy==1.26.0
nvidia-cublas-cu11==11.11.3.6
nvidia-cuda-cupti-cu11==11.8.87
nvidia-cuda-nvcc-cu11==11.8.89
nvidia-cuda-runtime-cu11==11.8

In [2]:
# import google.cloud.storage

# # Get the version of the google-cloud-storage library
# storage_version = google.cloud.storage.__version__

# print(f"Google Cloud Storage Client Library Version: {storage_version}")

Google Cloud Storage Client Library Version: 2.12.0


In [2]:
nest_asyncio.apply()
async def my_coroutine():
    app = FastAPI()

    origins = [
    "http://localhost/",
    "http://localhost:3000/",
]
    app.add_middleware(
        CORSMiddleware,
        allow_origins=origins,
        allow_credentials=True,
        allow_methods=[""],
        allow_headers=[""],
    )

    Model = tf.keras.models.load_model("../saved_models1/1")

    Class_names = ["Tomato_Bacterial_spot", "Tomato_Early_blight", "Tomato_Late_blight","Tomato_Leaf_Mold",
                    "Tomato_Septoria_leaf_spot","Tomato__Target_Spot","Tomato__Tomato_YellowLeaf__Curl_Virus","Tomato__Tomato_mosaic_virus","Tomato_healthy"]

    @app.get("/ping")
    async def ping():
        return "Hello, I am alive"
    

    def read_file_as_image(data) -> np.ndarray:
        image=np.array(Image.open(BytesIO(data)))
        return image

    @app.post("/predict")
    async def predict(
        file: UploadFile = File(...)
        ):
        image = read_file_as_image(await file.read())
        img_batch = np.expand_dims(image,0)

        prediction = Model.predict(img_batch)

        predicted_class = Class_names[np.argmax(prediction[0])]
        confidence=np.max(prediction[0])
        
        return{
            'class' : predicted_class,
            'confidence': float(confidence)
        }
    
    if __name__ == "__main__":
        uvicorn.run(app, host='localhost', port=8000)

loop = asyncio.get_event_loop()
loop.run_until_complete(my_coroutine())

2023-10-16 00:20:13.192277: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 00:20:13.438531: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 00:20:13.438567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 00:20:13.440371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 00:20:13.440398: I tensorflow/compile

INFO:     127.0.0.1:45878 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:45878 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:45870 - "GET /ping HTTP/1.1" 200 OK


2023-10-16 00:21:01.381442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


1/1 [==============================] - 14s 14s/step
INFO:     127.0.0.1:34564 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [667]
